# Classical shadows for the v2RDM
This notebook provides an initial attempt at true classical constraints on the spatial v2RDM, as described in Appendix C of my Master's thesis. The result is inaccurate for the moment, but the code is very much a first draft.

The steps are:
1. Get the groundstate of a molecule
2. Apply Clifford unitary to the state
3. Measure in the computational basis
4. Invert the map for Clifford unitaries to get a classical shadow
5. Aggregating the shadows
6. Estimate the target function using the aggregated shadows

In [14]:
import numpy as np
from qiskit.quantum_info import random_clifford
from src.molecule_helper import create_molecule, get_hamiltonian, get_groundstate, get_spatial_D2

In [5]:
def sample_bitstring_from_statevector(psi, num_samples=1, seed=None):
    rng = np.random.default_rng(seed)
    probs = np.abs(psi) ** 2
    assert np.isclose(np.sum(probs), 1.0), "Statevector must be normalized"

    d = len(psi)
    n = int(np.log2(d))
    bitstrings = [format(i, f"0{n}b") for i in range(d)]

    # sample according to probability distribution given by state
    samples = rng.choice(bitstrings, size=num_samples, p=probs)
    return samples


def clifford_inverse(U, bvec, n):
    # inverse mapping provided by Huang et al
    return (2 ** n + 1) * U.conj().T @ bvec @ bvec.conj().T @ U - np.identity(2 ** n)


def make_shadow(gs, n_qubits):
    # 1. We have already gotten our ground state

    # sample Clifford
    clifford = random_clifford(n_qubits)
    U = clifford.to_operator().data

    # 2. Apply Clifford
    psi_prime = np.asarray(U @ gs).flatten()

    # 3. Measure and store bitstring
    bstr = sample_bitstring_from_statevector(psi_prime)[0]
    idx = int(bstr, 2)
    bvec = np.zeros((2 ** n_qubits, 1), dtype=complex)
    bvec[idx] = 1.0

    # 4. Apply inverse mapping
    rho_estimate = clifford_inverse(U, bvec, n_qubits)
    return rho_estimate


In [6]:
# Test
mol = create_molecule('hydrogen', 1.0)
H = get_hamiltonian(mol)
_, gs = get_groundstate(H)

N_shadows = 10000
shadows = [make_shadow(gs, mol.n_qubits) for _ in range(N_shadows)]

In [9]:
from openfermion import FermionOperator, jordan_wigner, get_sparse_operator
from itertools import product

# 5. Aggregate shadows
ave_shadows = sum(shadows)/N_shadows # just using average for now

# 6. Estimate target function
r = mol.n_qubits
Dshadow = np.zeros((r, r, r, r), dtype=np.complex128)

for i, j, k, l in product(range(r), repeat=4):
    op = FermionOperator(f"{i}^ {j}^ {k} {l}", 1.0) # our target function is the 2-RDM operator using second quantization
    op = jordan_wigner(op)
    op_dense = get_sparse_operator(op, n_qubits=mol.n_qubits).todense()
    Dshadow[i, j, k, l] += np.trace(op_dense @ ave_shadows)

Dshadow = Dshadow.reshape(r**2, r**2)
Dshadow = Dshadow/N_shadows

### Detour: Spin-orb to spatial conversion

In [35]:
def spinorbital_to_spatial_2rdm(D_spin):
    """
    Convert a spin-orbital 2-RDM (4r**2, 4r**2) into a spatial 2-RDM (r**2, r**2)
    by summing over spin indices.
    """
    assert D_spin.shape[0] % 2 == 0, "Spin-orbital tensor must have even dimension"
    r2 = int(np.sqrt(D_spin.shape[0]))
    r =  r2 // 2

    D_spatial = np.zeros((r, r, r, r))
    D_spin_tensor = D_spin.reshape((r2, r2, r2, r2))

    for i in range(r):
        for j in range(r):
            for k in range(r):
                for l in range(r):
                    # Spin indices
                    ia, ib = 2*i, 2*i+1
                    ja, jb = 2*j, 2*j+1
                    ka, kb = 2*k, 2*k+1
                    la, lb = 2*l, 2*l+1

                    # αααα
                    term_aa = D_spin_tensor[ia, ja, ka, la]
                    # ββββ
                    term_bb = D_spin_tensor[ib, jb, kb, lb]
                    # αββα
                    term_ab = D_spin_tensor[ia, jb, kb, la]
                    # βααβ
                    term_ba = D_spin_tensor[ib, ja, ka, lb]

                    D_spatial[i, j, l, k] = 0.5 * (term_aa + term_bb + term_ab + term_ba)

    return D_spatial.reshape(r*r, r*r) # minus apparently necessary because of OpenFermion indexing

### Back to shadows...

In [36]:
# Compare to FCI 2-RDM
spatial_shadow = spinorbital_to_spatial_2rdm(Dshadow)
np.trace(spatial_shadow) # trace strangely small

/var/folders/rj/x04zff0d7_z87bgpc5cs298r0000gq/T/ipykernel_44735/4011304751.py:32: ComplexWarning: Casting complex values to real discards the imaginary part
  D_spatial[i, j, l, k] = 0.5 * (term_aa + term_bb + term_ab + term_ba)


0.000100572499999999

In [37]:
dfci = get_spatial_D2(mol)

In [38]:
np.round(spatial_shadow - dfci, 3) # some difference... not sure why

array([[-0.969,  0.   ,  0.   ,  0.173],
       [ 0.   , -0.   ,  0.   , -0.   ],
       [ 0.   ,  0.   , -0.   , -0.   ],
       [ 0.173, -0.   , -0.   , -0.031]])